In [1]:
#importing the libraries needed for ML
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv(r'..\Datasets\Parkinsons Train Data.csv', header = 0)
data.head()

,Jitter(local),"Jitter(local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, db)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,status
0,1.488,0.000090,0.900,0.794,2.699,8.334,0.779,4.517,4.609,6.802,...,142.229,187.576,160,159,0.006065,0.000416,0.000,0,0.000,1
1,0.728,0.000038,0.353,0.376,1.059,5.864,0.642,2.058,3.180,7.194,...,159.515,234.505,170,169,0.005181,0.000403,2.247,0,0.000,1
2,1.220,0.000074,0.732,0.670,2.196,8.719,0.875,4.347,5.166,7.548,...,146.445,211.442,1431,1427,0.006071,0.000474,10.656,1,0.178,1
3,2.502,0.000123,1.156,1.634,3.469,13.513,1.273,5.263,8.771,16.779,...,182.713,220.230,94,92,0.004910,0.000320,0.000,0,0.000,1
4,3.509,0.000167,1.715,1.539,5.145,9.112,1.040,3.102,4.927,12.823,...,182.821,225.162,117,114,0.004757,0.000380,18.182,1,13.318,1


In [3]:
#setting the dependent variable
y = data.iloc[:, -1].values
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [ ]:
X = data.iloc[:, :-1].values
X

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_X = StandardScaler()
standard_scaled_X = standard_X.fit_transform(X)
standard_scaled_X

In [ ]:
def standardScale(X):
    X = standard_X.transform(X)
    return X

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
minmax_scaled_X = min_max_scaler.fit_transform(X)
minmax_scaled_X

In [ ]:
def minmaxScale(X):
    X = min_max_scaler.transform(X)
    return X

In [ ]:
data.columns

In [16]:
fixed_cols = ['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods',
              'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks']
jitter_cols = ['Jitter(local)', 'Jitter(local, absolute)', 'Jitter (rap)','Jitter (ppq5)', 'Jitter (ddp)']
shimmer_cols = ['Shimmer (local)', 'Shimmer (local, db)', 'Shimmer (apq3)', 'Shimmer (apq5)', 'Shimmer (apq11)',
                'Shimmer (dda)']
pitch_cols = ['Median Pitch', 'Mean Pitch']

In [17]:
def getEvaluations(model, y, Y_test, X_test, X_train, Y_train):
    from sklearn.metrics import accuracy_score
    print("Accuracy for training set -> ", end = "")
    Y_hat = [round(yhat) for yhat in model.predict(X)]
    print(accuracy_score(y, Y_hat)) # Full set accuracy
    Y_hat = [round(yhat) for yhat in model.predict(X_test)]
    print("Accuracy for test set -> ", end = "")
    print(accuracy_score(Y_test, Y_hat)) # Test set accuracy

    #Validations
    y_pred = model.predict(X_test)

    #Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(Y_test, y_pred) # Calulate Confusion matrix for test set.
    print(cm)

    from sklearn.model_selection import cross_val_score
    accuracies = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = 10)
    print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
    print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [18]:
def getRandomForest(X, y):
    
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=12, test_size = 0.2)
    # Train
    from sklearn.ensemble import RandomForestClassifier

    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, Y_train)
    # Evaluate
    getEvaluations(model, y, Y_test, X_test, X_train, Y_train)
    
    print("\nMAX_DEPTH increased to 3\n")
    
    model_1 = RandomForestClassifier(max_depth=3, n_estimators=100, random_state=0)
    model_1.fit(X_train, Y_train)

    # Evaluate
    getEvaluations(model_1, y, Y_test, X_test, X_train, Y_train)

In [19]:
for jitter in jitter_cols:
    for shimmer in shimmer_cols:
        for pitch in pitch_cols:
            
            fixed_cols.append(jitter)
            fixed_cols.append(shimmer)
            fixed_cols.append(pitch)
            print(fixed_cols)
            print("\n")
            print("Added columns to the dataset are:", jitter, shimmer, pitch)
            print("\n")
            X = data[fixed_cols].values
            print("---------Standard Scaled---------")
            print("\n")
            standard_X = StandardScaler()
            standard_scaled_X = standard_X.fit_transform(X)
            standard_scaled_X
            getRandomForest(standard_scaled_X, y)
            print("\n")
            print("---------MinMax Scaled---------")
            print("\n")
            min_max_scaler = preprocessing.MinMaxScaler()
            minmax_scaled_X = min_max_scaler.fit_transform(X)
            minmax_scaled_X
            getRandomForest(minmax_scaled_X, y)
            fixed_cols.remove(jitter)
            fixed_cols.remove(shimmer)
            fixed_cols.remove(pitch)
            print("\n")

['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter(local)', 'Shimmer (local)', 'Median Pitch']


Added columns to the dataset are: Jitter(local) Shimmer (local) Median Pitch


---------Standard Scaled---------


Accuracy for training set -> 0.49903474903474904
Accuracy for test set -> 0.6875
[[67 39]
 [26 76]]
Accuracy: 69.82 %
Standard Deviation: 4.57 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5231660231660231
Accuracy for test set -> 0.6634615384615384
[[57 49]
 [21 81]]
Accuracy: 63.78 %
Standard Deviation: 4.21 %


---------MinMax Scaled---------


Accuracy for training set -> 0.5115830115830116
Accuracy for test set -> 0.6826923076923077
[[67 39]
 [27 75]]
Accuracy: 69.58 %
Standard Deviation: 4.47 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5096525096525096
Accuracy for test set -> 0.6634615

Accuracy: 63.29 %
Standard Deviation: 4.16 %


---------MinMax Scaled---------


Accuracy for training set -> 0.5386100386100386
Accuracy for test set -> 0.6826923076923077
[[71 35]
 [31 71]]
Accuracy: 66.80 %
Standard Deviation: 4.53 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.527027027027027
Accuracy for test set -> 0.6682692307692307
[[58 48]
 [21 81]]
Accuracy: 63.29 %
Standard Deviation: 4.16 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter(local)', 'Shimmer (apq11)', 'Median Pitch']


Added columns to the dataset are: Jitter(local) Shimmer (apq11) Median Pitch


---------Standard Scaled---------


Accuracy for training set -> 0.5704633204633205
Accuracy for test set -> 0.7355769230769231
[[72 34]
 [21 81]]
Accuracy: 69.45 %
Standard Deviation: 4.14 %

MAX_DEPTH increased to 3

Accuracy for training set

Accuracy for training set -> 0.5617760617760618
Accuracy for test set -> 0.6923076923076923
[[67 39]
 [25 77]]
Accuracy: 67.16 %
Standard Deviation: 6.03 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.527992277992278
Accuracy for test set -> 0.6730769230769231
[[56 50]
 [18 84]]
Accuracy: 63.05 %
Standard Deviation: 3.98 %


---------MinMax Scaled---------


Accuracy for training set -> 0.5
Accuracy for test set -> 0.6971153846153846
[[68 38]
 [25 77]]
Accuracy: 67.04 %
Standard Deviation: 5.91 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5
Accuracy for test set -> 0.6730769230769231
[[56 50]
 [18 84]]
Accuracy: 63.05 %
Standard Deviation: 3.98 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter(local, absolute)', 'Shimmer (apq3)', 'Median Pitch']


Added columns to the dataset are: Jitter(local, ab

Accuracy: 63.53 %
Standard Deviation: 4.29 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter(local, absolute)', 'Shimmer (dda)', 'Mean Pitch']


Added columns to the dataset are: Jitter(local, absolute) Shimmer (dda) Mean Pitch


---------Standard Scaled---------


Accuracy for training set -> 0.45656370656370654
Accuracy for test set -> 0.6923076923076923
[[69 37]
 [27 75]]
Accuracy: 67.28 %
Standard Deviation: 4.89 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5183397683397684
Accuracy for test set -> 0.6682692307692307
[[55 51]
 [18 84]]
Accuracy: 63.05 %
Standard Deviation: 3.78 %


---------MinMax Scaled---------


Accuracy for training set -> 0.49903474903474904
Accuracy for test set -> 0.6875
[[69 37]
 [28 74]]
Accuracy: 67.16 %
Standard Deviation: 5.27 %

MAX_DEPTH increased to 3

Accuracy for training 

Accuracy: 65.46 %
Standard Deviation: 3.84 %


---------MinMax Scaled---------


Accuracy for training set -> 0.500965250965251
Accuracy for test set -> 0.7067307692307693
[[72 34]
 [27 75]]
Accuracy: 68.36 %
Standard Deviation: 3.94 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.502895752895753
Accuracy for test set -> 0.6730769230769231
[[66 40]
 [28 74]]
Accuracy: 65.34 %
Standard Deviation: 3.84 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter (rap)', 'Shimmer (apq5)', 'Mean Pitch']


Added columns to the dataset are: Jitter (rap) Shimmer (apq5) Mean Pitch


---------Standard Scaled---------


Accuracy for training set -> 0.5057915057915058
Accuracy for test set -> 0.6730769230769231
[[70 36]
 [32 70]]
Accuracy: 67.40 %
Standard Deviation: 3.28 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.538

Accuracy for training set -> 0.4942084942084942
Accuracy for test set -> 0.6971153846153846
[[71 35]
 [28 74]]
Accuracy: 68.72 %
Standard Deviation: 3.61 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5463320463320464
Accuracy for test set -> 0.6971153846153846
[[64 42]
 [21 81]]
Accuracy: 64.01 %
Standard Deviation: 4.37 %


---------MinMax Scaled---------


Accuracy for training set -> 0.527027027027027
Accuracy for test set -> 0.6971153846153846
[[71 35]
 [28 74]]
Accuracy: 68.60 %
Standard Deviation: 3.51 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.502895752895753
Accuracy for test set -> 0.6971153846153846
[[64 42]
 [21 81]]
Accuracy: 64.01 %
Standard Deviation: 4.37 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter (ppq5)', 'Shimmer (local, db)', 'Mean Pitch']


Added columns to the dataset

Accuracy for training set -> 0.5
Accuracy for test set -> 0.6875
[[70 36]
 [29 73]]
Accuracy: 67.15 %
Standard Deviation: 5.71 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5424710424710425
Accuracy for test set -> 0.6875
[[64 42]
 [23 79]]
Accuracy: 63.89 %
Standard Deviation: 5.36 %


---------MinMax Scaled---------


Accuracy for training set -> 0.4980694980694981
Accuracy for test set -> 0.6826923076923077
[[69 37]
 [29 73]]
Accuracy: 67.27 %
Standard Deviation: 5.60 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5
Accuracy for test set -> 0.6875
[[64 42]
 [23 79]]
Accuracy: 63.89 %
Standard Deviation: 5.36 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter (ppq5)', 'Shimmer (dda)', 'Mean Pitch']


Added columns to the dataset are: Jitter (ppq5) Shimmer (dda) Mean Pitch


---------Standard Scale

Accuracy for training set -> 0.5
Accuracy for test set -> 0.7019230769230769
[[72 34]
 [28 74]]
Accuracy: 67.28 %
Standard Deviation: 3.20 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.5337837837837838
Accuracy for test set -> 0.6730769230769231
[[66 40]
 [28 74]]
Accuracy: 65.34 %
Standard Deviation: 3.80 %


---------MinMax Scaled---------


Accuracy for training set -> 0.5
Accuracy for test set -> 0.7019230769230769
[[72 34]
 [28 74]]
Accuracy: 67.16 %
Standard Deviation: 3.17 %

MAX_DEPTH increased to 3

Accuracy for training set -> 0.502895752895753
Accuracy for test set -> 0.6730769230769231
[[66 40]
 [28 74]]
Accuracy: 65.22 %
Standard Deviation: 3.79 %


['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods', 'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks', 'Jitter (ddp)', 'Shimmer (apq5)', 'Mean Pitch']


Added columns to the dataset are: Jitter (ddp) Shimmer (apq5) M

In [ ]:
fixed_cols